In [1]:
import tensorflow as tf
import os
import time
import numpy as np
import os

In [15]:
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
x_train = x_train[..., np.newaxis].astype(np.float32) / 255.0

In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=1)

C:\Users\indum\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 17s 8ms/step - accuracy: 0.8719 - loss: 0.4691


In [21]:
model.export("mnist_model") 

INFO:tensorflow:Assets written to: mnist_model\assets


INFO:tensorflow:Assets written to: mnist_model\assets


Saved artifact at 'mnist_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  2354188580432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2354188581776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2354188581392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2354188579472: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [23]:
def representative_data_gen():
    for i in range(100):
        yield [x_train[i:i+1]]

In [25]:
converter = tf.lite.TFLiteConverter.from_saved_model("mnist_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_model = converter.convert()

In [27]:
with open("mnist_model_int8.tflite", "wb") as f:
    f.write(tflite_model)
print("Quantized model size (KB):", os.path.getsize("mnist_model_int8.tflite") / 1024)

Quantized model size (KB): 29.84375


In [29]:
interpreter = tf.lite.Interpreter(model_path="mnist_model_int8.tflite")
interpreter.allocate_tensors()

In [33]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [35]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
test_image = x_test[0:1].astype(np.float32)  
test_image = np.expand_dims(test_image, axis=-1) 
test_image /= 255.0

In [37]:
input_scale, input_zero_point = input_details[0]['quantization']
test_image_int8 = test_image / input_scale + input_zero_point  
test_image_int8 = test_image_int8.astype(np.int8)

In [39]:
interpreter.set_tensor(input_details[0]['index'], test_image_int8)
start = time.time()
interpreter.invoke()  
end = time.time()

In [41]:
output_data = interpreter.get_tensor(output_details[0]['index'])
pred_label = np.argmax(output_data)

In [43]:
print(f"Prediction: {pred_label}")
print(f"Inference Time: {end - start:.4f} seconds")
saved_model_size = os.path.getsize("mnist_model/saved_model.pb") / 1024  
quantized_model_size = os.path.getsize("mnist_model_int8.tflite") / 1024  
print(f"Original SavedModel Size: {saved_model_size:.2f} KB")
print(f"Quantized Model Size: {quantized_model_size:.2f} KB")

Prediction: 7
Inference Time: 0.0251 seconds
Original SavedModel Size: 34.25 KB
Quantized Model Size: 29.84 KB
